In [1]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..") 
from gcforest.gcforest import GCForest
import xlwt

/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# 数据及参数

train 80%, test 20%

In [2]:
random_seed = 42
score = 'f1_weighted'
test_size = 0.2
train_size_arr = [1, 0.5, 0.1, 0.01]

def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = random_seed
    ca_config["max_layers"] = 20
    ca_config["early_stopping_rounds"] = 5
    ca_config["n_classes"] = 6
    ca_config["estimators"] = []
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "random_state" : random_seed})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "DecisionTreeClassifier"})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"})
    config["cascade"] = ca_config
    return config

In [3]:
path = os.getcwd()+'/../data/20122018freshwater_four_feature.csv'
data = pd.read_csv(path, na_values = np.nan)

X = data.drop(['本周水质'], axis=1).values # Series
y = data['本周水质'].values.reshape(-1,1) - 1

# 1. 中位数填充缺失值，2.Z-score标准化
clean_pipeline = Pipeline([('imputer', preprocessing.Imputer(missing_values='NaN',strategy="median")),
                           ('std_scaler', preprocessing.StandardScaler())])
X = clean_pipeline.fit_transform(X)

# Base Model Pre-train

In [4]:
config = get_toy_config()

models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    SVC(probability=True),
    DecisionTreeClassifier(),
    ExtraTreeClassifier(),
    GaussianNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(random_state=random_seed),
    ExtraTreesClassifier(random_state=random_seed),
    GCForest(config)
]

In [5]:
model_name_dic = {
    "LogisticRegression": "LR",
    "LinearDiscriminantAnalysis": "LDA",
    "SVC": "SVM",
    "DecisionTreeClassifier": "DT",
    "ExtraTreeClassifier":"CRT",
    "GaussianNB":"NB",
    "KNeighborsClassifier":"KNN",
    "RandomForestClassifier":"RF",
    "ExtraTreesClassifier":"CRTF",
    "GCForest": "DCF"
}

In [6]:
def train_exps(X, y, model, test_size, train_size_arr):
    model_name = model.__class__.__name__
    print(model_name_dic[model_name])

    n_samples = X.shape[0]
    n_features = X.shape[1]
    n_class = np.unique(y).shape[0]
    n_exps = len(train_size_arr)
    
    _Acc_matrix = np.zeros((n_class, n_exps))
    _Pr_matrix = np.zeros((n_class, n_exps))
    _Rc_matrix = np.zeros((n_class, n_exps))
    _F1_matrix = np.zeros((n_class, n_exps))
    _Support_matrix = np.zeros((n_class, n_exps))
    Acc_matrix = np.zeros((n_class, n_exps))
    Pr_matrix = np.zeros((n_class, n_exps))
    Rc_matrix = np.zeros((n_class, n_exps))
    F1_matrix = np.zeros((n_class, n_exps))
    Support_matrix = np.zeros((n_class, n_exps))

    # 数据划分
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                        stratify = y, random_state = random_seed)
    X_train_tmp = X_train
    y_train_tmp = y_train
    X_test = clean_pipeline.fit_transform(X_test)
    
    col = 0
    for k in train_size_arr:
        print("train_size:", k)
        _Acc_arr = np.zeros(n_class)
        _Pr_arr = np.zeros(n_class)
        _Rc_arr = np.zeros(n_class)
        _F1_arr = np.zeros(n_class)
        _Support_arr = np.zeros(n_class)
        Acc_arr = np.zeros(n_class)
        Pr_arr = np.zeros(n_class)
        Rc_arr = np.zeros(n_class)
        F1_arr = np.zeros(n_class)
        Support_arr = np.zeros(n_class)

        if k != 1:
            X_train, X_test_, y_train, y_test_ = train_test_split(X_train_tmp, y_train_tmp, test_size=1-k, stratify = y_train_tmp, random_state = random_seed)

        # 模型训练
        print(X_train.shape)
        if model_name == 'GCForest':
            model.fit_transform(X_train, y_train.reshape(y_train.shape[0]))
        else:
            model.fit(X_train, y_train)

        # 模型预测
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # 结果整理
        ## 由混淆矩阵计算各类别的Acc
        train_cm = confusion_matrix(y_train, y_train_pred)
        test_cm = confusion_matrix(y_test, y_test_pred)
        i = 0
        for c in test_cm:
            Acc_arr[i] = c[i]/np.sum(c)
            i += 1
        i = 0
        for c in train_cm:
            _Acc_arr[i] = c[i]/np.sum(c)
            i += 1
            
        ## 由classification_report提取各类别的precision, recall, F1
        cr = classification_report(y_test, y_test_pred, digits=4)
        _cr = classification_report(y_train, y_train_pred, digits=4)
        i = 0
        for l in range(2,8):
            Pr_arr[i] = float(cr.splitlines()[l].split()[1])
            _Pr_arr[i] = float(_cr.splitlines()[l].split()[1])
            Rc_arr[i] = float(cr.splitlines()[l].split()[2])
            _Rc_arr[i] = float(_cr.splitlines()[l].split()[2])
            F1_arr[i] = float(cr.splitlines()[l].split()[3])
            _F1_arr[i] = float(_cr.splitlines()[l].split()[3])
            Support_arr[i] = float(cr.splitlines()[l].split()[4])
            _Support_arr[i] = float(_cr.splitlines()[l].split()[4])
            i += 1
        Acc_matrix[:, col] = Acc_arr
        Pr_matrix[:, col] = Pr_arr
        Rc_matrix[:, col] = Rc_arr
        F1_matrix[:, col] = F1_arr
        Support_matrix[:, col] = Support_arr
        _Acc_matrix[:,col] = _Acc_arr
        _Pr_matrix[:, col] = _Pr_arr
        _Rc_matrix[:, col] = _Rc_arr
        _F1_matrix[:, col] = _F1_arr
        _Support_matrix[:, col] = _Support_arr
        col += 1
    
    return Acc_matrix, Pr_matrix, Rc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _Pr_matrix, _Rc_matrix, _F1_matrix, _Support_matrix

In [7]:
workbook = xlwt.Workbook(encoding = 'utf-8')

for model in models:
    model_name = model.__class__.__name__

    Acc_matrix, Pr_matrix, Rc_matrix, F1_matrix, Support_matrix, _Acc_matrix, _Pr_matrix, _Rc_matrix, _F1_matrix, _Support_matrix = train_exps(X, y, model, test_size, train_size_arr)
    # Acc, Pr, Rc 取平均, F1取加权平均
    Acc_mean = np.mean(Acc_matrix, axis = 0)
    Pr_mean = np.mean(Pr_matrix, axis = 0)
    Rc_mean = np.mean(Rc_matrix, axis = 0)
    WF1 = np.sum(F1_matrix*Support_matrix, axis = 0)/np.sum(Support_matrix, axis=0)
    _Acc_mean = np.mean(_Acc_matrix, axis = 0)
    _Pr_mean = np.mean(_Pr_matrix, axis = 0)
    _Rc_mean = np.mean(_Rc_matrix, axis = 0)
    _WF1 = np.sum(_F1_matrix*_Support_matrix, axis = 0)/np.sum(_Support_matrix, axis=0)
    
    worksheet = workbook.add_sheet(model_name_dic[model_name], cell_overwrite_ok=True)
    
    n_exps = len(train_size_arr)
    l=0
    worksheet.write(l,0, "Train Acc")
    worksheet.write(l,n_exps+2, "Test Acc")
    l+=1
    c=0
    for k in train_size_arr:
        worksheet.write(l,c, train_size_arr[c])
        worksheet.write(l,c+n_exps+2, train_size_arr[c])
        c+=1
    
    for i in range(6):
        l+=1
        for j in range(n_exps):
            worksheet.write(l, j, _Acc_matrix[i,j])
            worksheet.write(l, j+n_exps+2, Acc_matrix[i,j])
    l+=1
    for j in range(n_exps):
        worksheet.write(l, j, _Acc_mean[j])
        worksheet.write(l, j+n_exps+2, Acc_mean[j])
    
    #=======#
    l+=3
    worksheet.write(l,0, "Train Pr")
    worksheet.write(l,n_exps+2, "Test Pr")
    l+=1
    c=0
    for k in train_size_arr:
        worksheet.write(l,c, train_size_arr[c])
        worksheet.write(l,c+n_exps+2, train_size_arr[c])
        c+=1
    
    for i in range(6):
        l+=1
        for j in range(n_exps):
            worksheet.write(l, j, _Pr_matrix[i,j])
            worksheet.write(l, j+n_exps+2, Pr_matrix[i,j])
    l+=1
    for j in range(n_exps):
        worksheet.write(l, j, _Pr_mean[j])
        worksheet.write(l, j+n_exps+2, Pr_mean[j])

    #=======#
    l+=3
    worksheet.write(l,0, "Train Rc")
    worksheet.write(l,n_exps+2, "Test Rc")
    l+=1
    c=0
    for k in train_size_arr:
        worksheet.write(l,c, train_size_arr[c])
        worksheet.write(l,c+n_exps+2, train_size_arr[c])
        c+=1
    
    for i in range(6):
        l+=1
        for j in range(n_exps):
            worksheet.write(l, j, _Rc_matrix[i,j])
            worksheet.write(l, j+n_exps+2, Rc_matrix[i,j])
    l+=1
    for j in range(n_exps):
        worksheet.write(l, j, _Rc_mean[j])
        worksheet.write(l, j+n_exps+2, Rc_mean[j])
    #=======#
    l+=3
    worksheet.write(l,0, "Train F1")
    worksheet.write(l,n_exps+2, "Test F1")
    l+=1
    c=0
    for k in train_size_arr:
        worksheet.write(l,c, train_size_arr[c])
        worksheet.write(l,c+n_exps+2, train_size_arr[c])
        c+=1
    
    for i in range(6):
        l+=1
        for j in range(n_exps):
            worksheet.write(l, j, _F1_matrix[i,j])
            worksheet.write(l, j+n_exps+2, F1_matrix[i,j])
    l+=1
    for j in range(n_exps):
        worksheet.write(l, j, _WF1[j])
        worksheet.write(l, j+n_exps+2, WF1[j])

workbook.save("../res/各模型在不同数量的训练样本的比较.xls")

LR
train_size: 1
(26889, 4)


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train_size: 0.5
(13444, 4)
train_size: 0.1
(2688, 4)
train_size: 0.01
(268, 4)
LDA
train_size: 1
(26889, 4)
train_size: 0.5


/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(13444, 4)
train_size: 0.1


...

[ 2019-07-24 23:35:00,235][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_0.predict)=92.11%
[ 2019-07-24 23:35:00,258][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_1.predict)=91.27%
[ 2019-07-24 23:35:00,278][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_2.predict)=96.33%
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[ 2019-07-24 23:35:00,295][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_3.predict)=88.67%
[ 2019-07-24 23:35:00,317][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_4.predict)=88.49%
[ 2019-07-24 23:35:00,319][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_2 - estimator_2 - 5_folds.train_c

[ 2019-07-24 23:35:00,805][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_2 - 5_folds.train_4.predict)=90.23%
[ 2019-07-24 23:35:00,808][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_2 - 5_folds.train_cv.predict)=90.34%
[ 2019-07-24 23:35:00,831][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_3 - 5_folds.train_0.predict)=91.71%
[ 2019-07-24 23:35:00,850][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_3 - 5_folds.train_1.predict)=91.72%
[ 2019-07-24 23:35:00,871][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_3 - 5_folds.train_2.predict)=88.58%
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[ 2019-07-24 23:35:00,891][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_3 - estimator_3 - 5_folds.train_

[ 2019-07-24 23:35:01,527][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_3 - 5_folds.train_cv.predict)=91.82%
[ 2019-07-24 23:35:01,533][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_0.predict)=92.41%
[ 2019-07-24 23:35:01,536][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_1.predict)=88.86%
/Users/alex/anaconda/envs/gc/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[ 2019-07-24 23:35:01,539][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_2.predict)=88.85%
[ 2019-07-24 23:35:01,544][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_3.predict)=89.38%
[ 2019-07-24 23:35:01,548][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_4 - estimator_4 - 5_folds.train_

[ 2019-07-24 23:35:02,031][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_0.predict)=84.57%
[ 2019-07-24 23:35:02,036][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_1.predict)=84.46%
[ 2019-07-24 23:35:02,040][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_2.predict)=88.60%
[ 2019-07-24 23:35:02,044][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_3.predict)=87.20%
[ 2019-07-24 23:35:02,052][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_4.predict)=89.63%
[ 2019-07-24 23:35:02,054][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_5 - 5_folds.train_cv.predict)=86.99%
[ 2019-07-24 23:35:02,063][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_6 - 5_folds.train_0.predict)=83.35%
[ 2019-07-24 23:35:02,069][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_5 - estimator_6 - 5_folds

[ 2019-07-24 23:35:02,650][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_7 - 5_folds.train_1.predict)=93.90%
[ 2019-07-24 23:35:02,658][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_7 - 5_folds.train_2.predict)=90.26%
[ 2019-07-24 23:35:02,668][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_7 - 5_folds.train_3.predict)=90.55%
[ 2019-07-24 23:35:02,675][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_7 - 5_folds.train_4.predict)=93.37%
[ 2019-07-24 23:35:02,677][kfold_wrapper.log_eval_metrics] Weighted F1 (layer_6 - estimator_7 - 5_folds.train_cv.predict)=92.34%
[ 2019-07-24 23:35:02,682][cascade_classifier.calc_f1] Weighted F1 (layer_6 - train.classifier_average)=92.13%
[ 2019-07-24 23:35:02,684][cascade_classifier.fit_transform] [Result][Optimal Level Detected] opt_layer_num=2, weighted_f1_train=93.16%, weighted_f1_test=0.00%
[ 2019-07-24 23:35:02,687][cascade_classifier.transform] X_groups_test.shape=[(268, 4)]
